In [2]:
import numpy as np
import re
import string
import pandas as pd
import nltk
from nltk.collocations import *
from pattern.en import lemma
import requests
import json
import time
import hashlib
import uuid
import csv
import phrasemachine
import time
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.request import Request
import difflib
from nltk.stem import WordNetLemmatizer 
from wechatpy.client import WeChatClient
import random
import http.client
import stardict
from stardict import LemmaDB
import urllib

In [3]:
master_word_dict = {}

HP1_word_final = []
csvFile = open("HP1vocabularycounts_v2.5.csv", "r")
reader = csv.reader(csvFile)
for i in reader:
    HP1_word_final.append(i[0])
master_word_dict['HP1'] = HP1_word_final

HP2_word_final = []
csvFile = open("HP2vocabularycounts_v2.5.csv", "r")
reader = csv.reader(csvFile)
for i in reader:
    HP2_word_final.append(i[0])
master_word_dict['HP2'] = HP2_word_final

HP3_word_final = []
csvFile = open("HP3vocabularycounts_v2.csv", "r")
reader = csv.reader(csvFile)
for i in reader:
    HP3_word_final.append(i[0])
master_word_dict['HP3'] = HP3_word_final

HP4_word_final = []
csvFile = open("HP4vocabularycounts_v2.csv", "r")
reader = csv.reader(csvFile)
for i in reader:
    HP4_word_final.append(i[0])
master_word_dict['HP4'] = HP4_word_final

HP5_word_final = []
csvFile = open("HP5vocabularycounts_v2.csv", "r")
reader = csv.reader(csvFile)
for i in reader:
    HP5_word_final.append(i[0])
master_word_dict['HP5'] = HP5_word_final

HP6_word_final = []
csvFile = open("HP6vocabularycounts_v2.csv", "r")
reader = csv.reader(csvFile)
for i in reader:
    HP6_word_final.append(i[0])
master_word_dict['HP6'] = HP6_word_final

HP7_word_final = []
csvFile = open("HP7vocabularycounts_v2.csv", "r")
reader = csv.reader(csvFile)
for i in reader:
    HP7_word_final.append(i[0])
master_word_dict['HP7'] = HP7_word_final

Heartisthelonelyhunter_word_final = []
csvFile = open("The Heart is a Lonely Hunter_VocabCount_v4.csv", "r")
reader = csv.reader(csvFile)
for i in reader:
    Heartisthelonelyhunter_word_final.append(i[0])
master_word_dict['Heart is the Lonely Hunter'] = Heartisthelonelyhunter_word_final

Theroomwhereithappens_word_final = []
csvFile = open("Theroomwhereithappensvocabularycounts_v2.5.csv", "r")
reader = csv.reader(csvFile)
for i in reader:
    Theroomwhereithappens_word_final.append(i[0])
master_word_dict['The Room Where It Happends'] = Theroomwhereithappens_word_final

#### What I should do next? 
#### Consider what is a better way of lemma (Now I have to lemma the replicate list to get a better result and it should be fixed)***
#### Consider how to build a database or container


# 1.Clean the text, lemma words

In [ ]:
with open("TheHeartisthelonelyhuntervocabularycounts_v2.5.txt", 'r',encoding='utf-8') as file:
    text = file.read().rstrip().lower()

In [142]:
# remove puntuation
word_list = re.findall("[a-z]+\-?[a-z]+", text)

In [143]:
#Lemmatization
#LemmaDB can't lemma didn't or wouldn't, etc.
lemma_word_list = []
lemma = LemmaDB()
lemma.load('lemma.en.txt')

for i in range(0,len(word_list)):
    lemma_word = lemma.get(word_list[i], reverse = True)
    if lemma_word == None:
        lemma_word_list.append(word_list[i])
    else:
        lemma_word_list.append(lemma_word[0])

In [144]:
CET4_list = []

with open('CET-4_Vocab.txt','r',encoding='utf-8') as fhandle:
    for line in fhandle:
        if line.startswith(('a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z')):
            word = re.findall('([a-z]+)',line)
            CET4_list.append(word[0])
        else:
            continue

In [146]:
highschool_list = []

with open('Highschool_Vocab.txt','r',encoding='utf-8') as fhandle:
    for line in fhandle:
        highschool_list.append(line.lower().rstrip())    

In [147]:
elementary_list = []

with open('Elementary_Vocab.txt','r',encoding='utf-8') as fhandle:
    for line in fhandle:
        elementary_list.append(line.lower().rstrip())

In [149]:
easy_word_list = set(CET4_list) | set(highschool_list) | set(elementary_list)
easy_word_list = list(easy_word_list)


In [150]:
#select unfamiliar words
raw_word_list = []
for word in lemma_word_list:
    if word not in lemma_easy_word_list:
        raw_word_list.append(word)
    else:
        continue    

In [152]:
#count the word
word_count = pd.value_counts(raw_word_clean_list)
data = pd.DataFrame({'Vocabulary':word_count.index, 'counts':word_count.values})

In [153]:
# point out where the replicate comes from
def get_key (dict, value):
    return [k for k, v in dict.items() if v == value]

In [154]:
for i in range(0,len(word_count.index)):
    for j in list(master_word_dict.values()):
        if data.loc[i,'Vocabulary'] in j:
            data.loc[i,'Replicate'] = get_key(master_word_dict,j)
        else:
            pass
            
data.to_csv("Heartisthelonelyhuntervocabularycounts_v0.csv",index=False,header=True)            

# 2. Translation & Phonetics & Characteristic & Category

#### 2.1 test

In [ ]:
app_id = '761150e6'
app_key = '9c547ac3ff97fad8a7e366f8129d6a2c'

word_id = 'better'
strictMatch = 'false'

url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/en-gb/' + word_id.lower() + '?strictMatch=' + strictMatch;

r = requests.get(url, headers = {'app_id': app_id, 'app_key': app_key})
pron = r.json()["results"][0]['lexicalEntries'][0]['entries'][0]['pronunciations'][0]['phoneticSpelling']
cate = r.json()["results"][0]['lexicalEntries'][0]['lexicalCategory']['text']
defin = r.json()["results"][0]['lexicalEntries'][0]['entries'][0]['senses'][0]['definitions'][0]
print(pron,cate,defin)

In [ ]:
a = r.json()
b = a["results"][0]['lexicalEntries'][0]['entries'][0]['pronunciations'][0]['phoneticSpelling']


In [ ]:
app_id = '5c864748'
app_key = '11013df45a50b3d7960bdcb4fe77af01'

language = 'en'
word_id = 'better'
strictMatch = 'false'

url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/' + language + '/' + word_id.lower() + '?fields=' + fields + '&strictMatch=' + strictMatch;

r = requests.get(url, headers = {'app_id': app_id, 'app_key': app_key})
pron = r.json()
pron

In [ ]:
c = r.json()
d = c["results"][0]['lexicalEntries'][0]['lexicalCategory']['text']
d

In [ ]:
# Youdao translation for plan B
youdao_url = 'https://openapi.youdao.com/api'  

input_text = "better"
translate_text = "dream"
time_curtime = int(time.time())   # 秒级时间戳获取
app_id = "15a1da979e8d68c0"   # 应用id
uu_id = uuid.uuid4()   # 随机生成的uuid数，为了每次都生成一个不重复的数。
app_key = "SSfsA7MhSvqvG79kKVzGx875O2ukapUW"   # 应用密钥

sign = hashlib.sha256((app_id + input_text + str(uu_id) + str(time_curtime) + app_key).encode('utf-8')).hexdigest()   # sign生成


data = {
    'q':translate_text,   # 翻译文本
    'from':"en",   # 源语言
    'to':"zh-CHS",   # 翻译语言
    'appKey':app_id,   # 应用id
    'salt':uu_id,   # 随机生产的uuid码
    'sign':sign,   # 签名
    'signType':"v3",   # 签名类型，固定值
    'curtime':time_curtime,   # 秒级时间戳
}



r = requests.get(youdao_url, params = data).json()
r

#### 2.2 the full version

In [73]:
csvFile = open("Theroomwhereithappenedvocabularycounts_v2.csv", "r",encoding = 'utf-8')
reader = csv.reader(csvFile)

In [76]:
# API id and key
app_id = '05d6c109'
app_key = '63bda9528e24dff26ca5dd695b008acf'

In [77]:
GMAT_list = []

with open('GMAT_edited.txt','r',encoding='utf-8') as fhandle:
    for line in fhandle:
        GMAT_list.append(line.lower().rstrip())

In [22]:
word_pron_char_defin_cate = []   
df = pd.read_csv("stardict.csv",encoding = 'utf-8')

D:\apps\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [78]:
for i in reader:
    try:
        category = re.findall("[a-z]+\-?[a-z]+", df.loc[df['word']== i[0] ,'tag'].to_string())
        if i[0] in GMAT_list:
            category.append('gmat')
        cate = ",".join(category)
    
        try:
    
            word_id = i[0]
            strictMatch = 'false'

            url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/en-gb/' + word_id.lower() + '?strictMatch=' + strictMatch;

            r = requests.get(url, headers = {'app_id': app_id, 'app_key': app_key})
            pron = r.json()["results"][0]['lexicalEntries'][0]['entries'][0]['pronunciations'][0]['phoneticSpelling']
            char = r.json()["results"][0]['lexicalEntries'][0]['lexicalCategory']['text']
            defin = r.json()["results"][0]['lexicalEntries'][0]['entries'][0]['senses'][0]['definitions'][0]
            word_pron_char_defin_cate.append([i[0],'/'+pron+'/',char,defin,cate])
            time.sleep(10)
        except KeyError :
            pass
        continue
    except IndexError:
        pass

IndexError: list index out of range

In [81]:
word_pron_char_defin_cate

[['sanction',
  '/ˈsaŋ(k)ʃ(ə)n/',
  'Noun',
  'a threatened penalty for disobeying a law or rule',
  'cet,ky,toefl,ielts,gre,gmat'],
 ['regime',
  '/reɪˈʒiːm/',
  'Noun',
  'a government, especially an authoritarian one',
  'cet,ky,toefl,ielts,gre,gmat'],
 ['treasury',
  '/ˈtrɛʒ(ə)ri/',
  'Noun',
  'the funds or revenue of a state, institution, or society',
  'gmat'],
 ['bilateral',
  '/bʌɪˈlat(ə)r(ə)l/',
  'Adjective',
  'having or relating to two sides; affecting both sides',
  'toefl,ielts,gmat'],
 ['capability',
  '/keɪpəˈbɪlɪti/',
  'Noun',
  'the power or ability to do something',
  'cet,toefl'],
 ['impeachment',
  '/ɪmˈpiːtʃmənt/',
  'Noun',
  '(especially in the US) a charge of misconduct made against the holder of a public office',
  ''],
 ['tariff',
  '/ˈtarɪf/',
  'Noun',
  'a tax or duty to be paid on a particular class of imports or exports',
  'cet,ky,toefl,ielts,gre,gmat'],
 ['concession',
  '/kənˈsɛʃ(ə)n/',
  'Noun',
  'a thing that is granted, especially in response to

In [89]:
for i in word_pron_char_defin_cate:
    if i[2] == 'Noun':
        i[2] = 'n.'
    if i[2] == 'Adjective':
        i[2] = 'adj.'
    if i[2] == 'Verb':
        i[2] = 'v.'
    if i[2] == 'Adverb':
        i[2] = 'adv.'


In [90]:
# Baidu API for EN-CN Translate
appid = '20200627000507736'  # 填写你的appid
secretKey = 'G3ghonyD9dfI39NnFtGN'  # 填写你的密钥

httpClient = None
myurl = '/api/trans/vip/translate'

fromLang = 'en'   #原文语种
toLang = 'zh'   #译文语种

In [91]:
len(word_pron_char_defin_cate)

100

In [92]:
word_pron_char_defin_cate_cn = []
for i in word_pron_char_defin_cate:
    salt = random.randint(32768, 65536)
    sign = appid + i[0] + str(salt) + secretKey
    sign = hashlib.md5(sign.encode()).hexdigest()
    myurl = myurl + '?appid=' + appid + '&q=' + urllib.parse.quote(i[0]) + '&from=' + fromLang + '&to=' + toLang + '&salt=' + str(salt) + '&sign=' + sign

    try:
        httpClient = http.client.HTTPConnection('api.fanyi.baidu.com')
        httpClient.request('GET', myurl)

        # response是HTTPResponse对象
        response = httpClient.getresponse()
        result_all = response.read().decode("utf-8")
        result = json.loads(result_all)['trans_result'][0]['dst']
        i.append(result)
        word_pron_char_defin_cate_cn.append(i)


    except Exception as e:
        pass



In [93]:
fileObject = open('Theroomwhereithappensvocabularycounts_v3.txt', 'w',encoding='utf-8')
for i in word_pron_char_defin_cate_cn:
    for j in i:
        fileObject.write(j)
        fileObject.write(' ')
    fileObject.write('\n')
fileObject.close()

# 3. Find phrases in text

#### 3.1 test

In [ ]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = BigramCollocationFinder.from_words(word_list)
scored = finder.score_ngrams(bigram_measures.raw_freq)
sorted(bigram for bigram, score in scored)

In [ ]:
text = "The Omnibus Crime Control and Safe Streets Act of 1968 was signed into law by President Lyndon B. Johnson.so as James"
phrasemachine.get_phrases(text)

In [ ]:
phrase_list=[['bring','out'],['look','in','the','eye'],['dream','of'],['want','to'],['had','it','in']]
raw_phrase_list = []

#### 3.2 full version

In [97]:
# Pharase with space
csvFile = open("Phrase Book With Space.csv", "r",encoding = 'utf-8-sig')
reader = csv.reader(csvFile)
phrase = []
phrase_with_space_list = []
for i in reader:
    phrase.append(i)
for i in phrase:
    i = [j for j in i if j !=  '']
    phrase_with_space_list.append(i)

In [99]:
# select pharses appeared with spaces
raw_phrase_with_space_list = []

for i in range(0,len(word_list)):
    for j in phrase_with_space_list:
        try:
            if word_list[i] == j[0] and j[-1] in word_list[i+1:i+5]:
                if j[-2] in word_list[i+1:i+4]:
                    raw_phrase_with_space_list.append(j)
                else:
                    continue                
            else:
                continue
        except IndexError:
            pass
        continue

phrase_count = pd.value_counts(raw_phrase_with_space_list)
data = pd.DataFrame({'Phrase':phrase_count.index, 'counts':phrase_count.values})
data.to_csv("Heartisthelonelyhunter_phrase_with_space_counts.csv",index=False,header=True)  

In [100]:
# Pharase without space
csvfile = open("Phrase Book No Space_v2.csv", "r",encoding = 'utf-8-sig')
reader = csv.reader(csvfile)
phrase = []
phrase_no_space_list = []
for i in reader:
    phrase.append(i)
for i in phrase:
    i = [j for j in i if j !=  '']
    phrase_no_space_list.append(i)

In [102]:
# select pharses appeared without spaces
raw_phrase_no_space_list = []
correct = 0
for i in range(0,len(word_list)):
    for j in phrase_no_space_list:
        try:
            length = len(j)
            for h in range(0,length):
                if word_list[i+h] == j[h]:
                    correct += 1
            if correct == length:
                raw_phrase_no_space_list.append(j)
                correct = 0
            else:
                correct = 0
        except IndexError:
            pass
        continue
            
phrase_count = pd.value_counts(raw_phrase_no_space_list)
data = pd.DataFrame({'Phrase':phrase_count.index, 'counts':phrase_count.values})
data.to_csv("Heartisthelonelyhunter_phrase_no_space_counts.csv",index=False,header=True)
        
            

# 4. Scrape Booklist

#### bookerprize

In [65]:
for i in raw_phrase_no_space_list:
    raw_phrase_with_space_list.append(i)

In [66]:
phrase_count = pd.value_counts(raw_phrase_with_space_list)
data = pd.DataFrame({'Phrase':phrase_count.index, 'counts':phrase_count.values})
data.to_csv("Roomwhereithappendsphrasecounts.csv",index=False,header=True)

In [ ]:
url = []
for i in range(2010,2020):
    url.append('https://thebookerprizes.com/fiction/backlist/'+str(i))

In [ ]:
name_author_list = []
for i in url:
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(i,headers=hdr)
    html= urlopen(req)
    soup = BeautifulSoup(html, 'html.parser')
    shortlist = soup.find('div',class_="flexslider bookCarousel")
    name_author=shortlist.find_all('h4',class_="title")
    for k in name_author:
        try:
            name = k.get_text()
            author = re.findall('\-by\-(\S+)',k.a['href'])[0].replace('-',' ')
            name_author_list.append([name,author])
        except IndexError:
            pass
        continue

In [ ]:
name_author_list

In [ ]:
word = 'https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcaJnXUm70lia7Ot4h9L21OZFUzeNic40wicevPiajFuXlf3K65PNgiaPbtzg/0?wx_fmt=jpeg'
defination = 'https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcxUoRJ1YErqXYmesqTaxPnpJ7icDbyLat6tDUlQIeHFpo0fEdOcXJtFw/0?wx_fmt=jpeg'
css = """<section data-width="100%" style="display: inline-block; width: 100%; font-size: -3px; background-image: url(&quot;""" + defination + """&quot;); background-size: cover; vertical-align: top; background-attachment: scroll; box-sizing: border-box; text-align: center; background-position: 0% 0%; background-repeat: no-repeat; max-width: 100% !important;">
<svg data-width="100%" enable-background="new 0 0 1067 212" space="preserve" style="display: inline-block;outline: none;width: 100%;margin-top: -1px;border-width: initial;border-style: none;border-color: initial;vertical-align: top;background-position: 0% 0%;background-repeat: no-repeat;background-size: 100%;background-attachment: scroll;background-image: url(&quot;""" + word + """&quot;);-webkit-tap-highlight-color: transparent;" viewBox="0 0 1067 212" x="0px" xml="" y="0px">
<set attributeName="visibility" from="visible" to="hidden" begin="click+0.5"></set>
<animate attributeName="opacity" begin="click" dur="0.5s" values="1;0" restart="never" fill="freeze">
</animate>
<set attributeName="visibility" from="hidden" to="visible" begin="click+3"></set>
<animate attributeName="opacity" begin="click+3" dur="0.3s" values="0;1" restart="never" fill="freeze">
</animate>
</svg></section>"""
print(css)

# 5. CSS editor

In [7]:
app = {
    "appid": "wx72b0cc170a66fe7b",
    "appsecret": "9e9ef69b54fd1688376dc82b130f4bf9"
}
wx = WeChatClient(app.get("appid"), app.get("appsecret"))

In [36]:
pic_title_link = "D://FC/Title.JPG"
pic_bottom_link = "D://FC/Bottom.JPG"
pic_title_url =  wx.material.add("image",open(pic_title_link,"rb"))['url']
pic_bottom_url =  wx.material.add("image",open(pic_bottom_link,"rb"))['url']

In [35]:
pic_word_link = []
pic_defin_link = []

for i in range (1,101):
    if i % 2 != 0:
        pic = "D://FC/幻灯片" + str(i) + ".JPG"
        pic_word_link.append(pic)
    else:
        pic = "D://FC/幻灯片" + str(i) + ".JPG"
        pic_defin_link.append(pic)



In [21]:
pic_word_url = []
pic_defin_url = []

for i in pic_word_link:
    resp = wx.material.add("image",open(i,"rb"))
    pic_word_url.append(resp['url'])


for i in pic_defin_link:
    resp = wx.material.add("image",open(i,"rb"))
    pic_defin_url.append(resp['url'])
    


In [29]:
pic_html = []


pic_html.append("<img src=" + pic_title_url + """data-type="jpg" style="float: left;">""")



for i in range(0,50):
    css = """<section data-width="100%" style="display: inline-block; width: 100%; font-size: 0px; background-image: url(&quot;""" + pic_defin_url[i] + """&quot;); background-size: cover; vertical-align: top; background-attachment: scroll; box-sizing: border-box; text-align: center; background-position: 0% 0%; background-repeat: no-repeat; max-width: 100% !important;">
<svg data-width="100%" enable-background="new 0 0 1067 212" space="preserve" style="display: inline-block;outline: none;width: 100%;margin-top: -1px;border-width: initial;border-style: none;border-color: initial;vertical-align: top;background-position: 0% 0%;background-repeat: no-repeat;background-size: 100%;background-attachment: scroll;background-image: url(&quot;""" + pic_word_url[i] + """&quot;);-webkit-tap-highlight-color: transparent;" viewBox="0 0 1067 212" x="0px" xml="" y="0px">
<set attributeName="visibility" from="visible" to="hidden" begin="click+0.3"></set>
<animate attributeName="opacity" begin="click" dur="0.2s" values="1;0" restart="never" fill="freeze">
</animate>
<set attributeName="visibility" from="hidden" to="visible" begin="click+3"></set>
<animate attributeName="opacity" begin="click+3" dur="0.2s" values="0;1" restart="never" fill="freeze">
</animate>
</svg>
</section>""" 
    pic_html.append(css)
    
pic_html.append("<img src=" + pic_bottom_url + """data-type="jpg" >""")

In [31]:
fileObject = open('Wechat HTML.txt', 'w',encoding='utf-8')
for i in pic_html:
    fileObject.write(i)
    fileObject.write('\n')
fileObject.close()


# Delete Images

In [46]:
media_id_list = []
resp = wx.material.batchget(media_type="image",offset=0,count=20)
for i in resp['item']:
    media_id_list.append(i['media_id'])
for i in media_id_list:
    wx.material.delete(i)

# 6. Similar words

#### test

In [58]:
df_lite = pd.read_csv("ecdict.csv",encoding = 'utf-8')

D:\apps\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [67]:
globe_word = []
for i in df_lite['word']:
    i = str(i)
    lemma_word = lemma.get(i, reverse = True)
    if lemma_word == None:
        globe_word.append(i)
    else:
        globe_word.append(lemma_word)

In [71]:
difflib.get_close_matches('bust', globe_word,20,0.6)

['buist',
 'blust',
 'baust',
 'bst',
 'bustin',
 'bustee',
 'bustad',
 'bursty',
 'bedust',
 'wust',
 'usts',
 'ustr',
 'ustc',
 'usct',
 'obst',
 'just',
 'iust',
 'hust',
 'fust',
 'dbut']

In [ ]:
csvFile = open("The Heart is a Lonely Hunter_VocabCount_v4.csv", "r")
reader = csv.reader(csvFile)
match_word_list = []

In [34]:
len(globe_word)

3402564

In [ ]:
for i in reader:
    match_word = difflib.get_close_matches(i[0], set(word_list),10,0.2)
    match_word_list.append(match_word)   

In [ ]:
with open("The Heart is a Lonely Hunter_VocabCount_v7.csv","w",newline = '') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(match_word_list)

In [39]:
difflib.get_close_matches('prop', ['plup','prop','happy','pp','plop','pulp'],10,0.2)

['prop', 'plop', 'pp', 'pulp', 'plup', 'happy']

In [72]:
word = 'happy'
word = list(word)
for i in range(0,len(word)):
    for j in df['word']:
        for k in range(0,len(list(j))):
            if word[i] == list(j)[k]:
                i += 1
                k
                
                      

['h', 'a', 'p', 'p', 'y']

# 8. StarDict

In [ ]:
import stardict
from stardict import LemmaDB
lemma = LemmaDB()
lemma.load('lemma.en.txt')

In [ ]:
l = lemma.word_stem("hasn't")
print(l[0])

# 9. wordlist 50 words

In [ ]:
df = pd.read_csv("stardict.csv",encoding = 'utf-8')

In [ ]:
df.loc[df['word']=='abandon','tag'].to_string()

In [ ]:
csvfile = open("The Heart is a Lonely Hunter_VocabCount_v4.csv", "r")
reader = csv.reader(csvfile)

In [ ]:
word_category = []
for i in reader:
    category = re.findall("[a-z]+\-?[a-z]+", df.loc[df['word']== i[0] ,'tag'].to_string())
    if i[0] in GMAT_list:
        category.append('gmat')
    word_category.append(category)

In [ ]:
word_category

In [ ]:
import json
import pickle
json_handle = open('PEPChuZhong7_1.json', 'rb')
json_string = json_handle.read()
json_dict = json.loads(json_string)
type(json_dict)


# 10. Remove Replicates

In [47]:
master_word_list = []
with open("HPmaster word list.txt", 'r',encoding='utf-8') as file:
    for i in file:
        master_word_list.append(i.rstrip())
master_word_set = set(master_word_list)
master_word_list = list(master_word_set)

In [51]:
HP7_word_clean = []

csvFile = open("HP7vocabularycounts_v4.csv", "r")
reader = csv.reader(csvFile)
for i in reader:
    if i not in master_word_list:
        HP7_word_clean.append(i)
    else:
        pass


In [56]:
with open("HPVocabularycounts_v5.csv","w",newline = '') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(HP7_word_clean)